# Twitter sentiment analysis using AutoML

In [ ]:
34.255.197.32

In [ ]:
!pip install requests tabulate future matplotlib

In [ ]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

In [1]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu222.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu222.04, mixed mode, sharing)
  Starting server from /home/barteus/Work/DSV/kubeflow-examples/venv2/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp79exyc2f
  JVM stdout: /tmp/tmp79exyc2f/h2o_barteus_started_from_python.out
  JVM stderr: /tmp/tmp79exyc2f/h2o_barteus_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_barteus_m2w28d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [ ]:
df = h2o.import_file("training.1600000.processed.noemoticon.csv", col_names=["polarity", "id", "date", "query", "user", "text"])

In [ ]:
df

In [ ]:
# filter unknown Age and split 70% train and 30% test
train, test = df.split_frame(ratios=[.7])

In [ ]:
# Identify predictors and response
x = train.columns
y = "polarity"
x.remove(y)

In [ ]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

train["query"] = train["query"].asfactor()
test["query"] = test["query"].asfactor()

In [ ]:
train.summary()

In [ ]:
aml = H2OAutoML(max_runtime_secs = 60, balance_classes=True, max_after_balance_size=0.3)
aml.train(x = x, y = y, training_frame = train, validation_frame = test)

In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

In [ ]:
preds = aml.leader.predict(test)
preds

In [ ]:
aml.leader.params

In [ ]:
model_path = h2o.save_model(model=aml.leader, path="./mymodel", force=True)

In [ ]:
h2o.load_model(model_path)